In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
import seaborn as sns 
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8
LABELS = ["Normal", "Fraud"]

RANDOM_SEED = 42

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# подгружаем датасет 
df = pd.read_csv('../input/creditcardfraud/creditcard.csv')

In [7]:
df.head()

In [8]:
df.info()

Видим, что данные чистые - нет ни пропусков, ни категориальных переменных, которые бы требовали перекодировки в числовые признаки.

In [9]:
df.describe()

In [10]:
params = {'axes.titlesize':'32',
          'xtick.labelsize':'24',
          'ytick.labelsize':'24'}
plt.rcParams.update(params)
df.hist(figsize=(50, 30))

In [11]:
sns.heatmap(df.corr())

На гистограммах можно увидеть, что фактически все признаки имеют распределение близкое к нормальному.

In [12]:
y = df.Class
X = df.drop(columns = ['Class'],axis = 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [14]:
print(X_train.shape)
print(X_test.shape)

In [15]:
frauds = df[df.Class == 1]
normal = df[df.Class == 0]

In [16]:
frauds.shape

In [17]:
normal.shape

Данные распределены по классам очень неравномерно (мошеннические транзакции составляют менее 0.2% от общего количества транзакций, представленных в датасете)

In [18]:
frauds.Amount.describe()

In [19]:
normal.Amount.describe()

In [20]:
# стандартизируем исходные данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.values)
X_test = scaler.transform(X_test.values)

## Логистическая регрессия

Для начала построим наивную модель с использованием логистической регрессии

In [21]:
clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_train, y_train)
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

на выходе получили значение метрики ROC_AUC = 0.97

In [22]:
conf_matrix = confusion_matrix(y_test, clf.predict(X_test))

plt.figure(figsize=(10, 10))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

## модель Catboost

In [23]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         )

In [24]:
roc_auc_score(y_test, model.predict(X_test))

Catboost показал результат чуть лучше, чем логистическая регрессия - ROC_AUC = 0.98

## Stacking
Применим такой ансамблевый метод как стекинг, в который войдут модели KNeighbors, DecisionTree и LogisticRegression.

In [25]:
level0 = list()
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('lr', LogisticRegression()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
stack_model = StackingClassifier(estimators=level0, final_estimator=level1, n_jobs=-1, cv=5)
stack_model.fit(X_train, y_train)

In [26]:
roc_auc_score(y_test, stack_model.predict(X_test))

Значение метрики получилось хуже, чем у логистической регрессии и catboost'а.

## Deep Learning

На практике при поиске фрода или выявления аномалий чаще всего приходится иметь дело с неразмеченными данными.
В таком случае нам приходится обучать модели без учителя и соответственно решать задачу поиска аномалий.
Попробуем обучить Autoencoder на тестовых данных и оценить его работу на тестовой выборке с учетом того, что данные у нас все-таки размеченные и мы можем получить количественную оценку работы модели.

In [27]:
input_dim = X_train.shape[1]
encoding_dim = 22

input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)

decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

In [28]:
nb_epoch = 160
batch_size = 32

autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

In [29]:
autoencoder = load_model('model.h5')

In [30]:
predictions = autoencoder.predict(X_test)

In [31]:
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})

In [32]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

In [33]:
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

In [34]:
threshold = 10

In [35]:
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [36]:
y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, y_pred)

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

Значение метрики ROC_AUC = 0.96, что тоже довольно неплохо. Данная модель может неплохо справляется с решением задачи, если в реальности нам приходится иметь дело с неразмеченными данными.